# Using Reddit's API for Predicting Comments

### Scraping Thread Info from Reddit.com

Imported Libraries

In [1]:
import requests
import json
import time
import csv
import pandas as pd
import sklearn
import numpy as np
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix

/anaconda2/envs/dsi/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


We retrieve the information from reddit.com here. 

In [3]:
URL = "http://www.reddit.com/r/nba.json"

In [4]:
URL_2 = "https://www.reddit.com/r/dogs.json"

In [5]:
## YOUR CODE HERE
res = requests.get(URL)
res = requests.get(URL, headers={'User-agent': 'YOUR NAME Bot 0.1'})

In [6]:
res2 = requests.get(URL_2)
res2 = requests.get(URL_2, headers={'User-agent': 'Vijay Bot 0.1'})

In [7]:
data = res.json()

In [8]:
data2 = res2.json()

#### Getting more results
By default, Reddit will give you the top 25 posts. However, it is good to look at other features of the reddit data. 

In [9]:
## YOUR CODE HERE
print(data['data'].keys())

dict_keys(['modhash', 'dist', 'children', 'after', 'before'])


In [10]:
print(data2['data'].keys())

dict_keys(['modhash', 'dist', 'children', 'after', 'before'])


Looking at both datasets, Reddit provides the same columns. However, it is important to go through all the reddit posts provided. This is where we need to look at the after column. 

In [11]:
after = 't3_9aq4ns'
new_url = 'http://www.reddit.com/r/nba.json?after='+after
res = requests.get(URL, headers={'User-agent': 'Vijay Bot 0.1'})

In [12]:
after = 't3_9aq4ns'
new_url = 'http://www.reddit.com/r/dogs.json?after='+after
res2 = requests.get(URL, headers={'User-agent': 'Vijay Bot 0.1'})

In [13]:
new_data = res.json()
new_data['data']['children'].__len__()

27

The code below will read through the sub-reddit nba and download a series of nba reddit posts. 
Reddit will limit the number of requests per second you're allowed to make and for this reason and for this reason, it is important to end the code with time.sleep(3). 

In [15]:
all_posts = []
url = 'http://www.reddit.com/r/nba.json?after='
#each of these requests below does 26 and then 25 posts
for i in range(28):
    res = requests.get(url, headers={'User-agent': 'vijay'})
    data = res.json()
    for i in range(25):
        post = []
        post.append(data['data']['children'][i]['data']['title'])
        post.append(data['data']['children'][i]['data']['selftext'])
        post.append(data['data']['children'][i]['data']['subreddit'])
#     post.append(data['data']['children']['title'])
#     post.append(data['data']['children']['subreddit_name_prefixed'])
#     all_posts.append(post)
        all_posts.append(post)
    after = data['data']['after']
    url = 'http://www.reddit.com/r/nba.json?after=' + after
    
    #print('The current after: ', after)
    time.sleep(3)

The code below will read through the sub-reddit dogs and download a series of dogs reddit posts.

In [16]:
url = 'http://www.reddit.com/r/dogs.json?after='
#each of these requests below does 26 and then 25 posts
for i in range(28):
    res2 = requests.get(url, headers={'User-agent': 'vijay'})
    data2 = res2.json()
    for i in range(25):
        post = []
        post.append(data2['data']['children'][i]['data']['title'])
        post.append(data2['data']['children'][i]['data']['selftext'])
        post.append(data2['data']['children'][i]['data']['subreddit'])
#     post.append(data['data']['children']['title'])
#     post.append(data['data']['children']['subreddit_name_prefixed'])
#     all_posts.append(post)
        all_posts.append(post)
    after = data2['data']['after']
    url = 'http://www.reddit.com/r/dogs.json?after=' + after
    
    #print('The current after: ', after)
    time.sleep(3)

### Save your results as a CSV
Below, we will save the extracted information from reddit.com in a .csv file. Extracting information from reddit.com can be a rather time-consuming process and saving the results in a .csv file can allow for faster text mining and faster data analysis of the text. 

In [17]:
df = pd.DataFrame(all_posts, columns = ['Title','Text','Category'])
df.to_csv("scraped-data.csv", sep='\t')

Below, I will create two vectorizers: the count vectorizer and the term-frequency inverse document frequency vectorizer. What are these two things and why are they important? 
The count vectorizer simply counts the word frequencies. 
The term-frequency inverse document frequency vectorizer also counts word frequencies but it adjusts for words that occur more frequently. Such words include "the", "and", "but" that occur frequently but do not really add much value to the meaning of the post. 

In [4]:
vectorizer = CountVectorizer(input='./scraped-data.csv', stop_words=sklearn.feature_extraction.text.ENGLISH_STOP_WORDS)
data = pd.read_csv('scraped-data.csv',sep='\t')

In [5]:
tfidf_vectorizer = TfidfVectorizer(input='./scraped-data.csv', stop_words = sklearn.feature_extraction.text.ENGLISH_STOP_WORDS)
data = pd.read_csv('scraped-data.csv',sep='\t')
data["Category"] = data["Category"].map(lambda x: 'dogs' in x)*1

In [6]:
output = vectorizer.fit_transform(data.Text.values.astype('U'))

In [7]:
tfidf_output = tfidf_vectorizer.fit_transform(data.Text.values.astype('U'))

The data in both output and tfidf_output need to be parsed into a format that can be readable by the train-test-split model, which is the next step in our process. For this reason, both are converted into arrays. 

In [8]:
#to convert to a more readable format
train_data_features = output.toarray()
train_data_features_tfidf = tfidf_output.toarray()

In [9]:
vocab = vectorizer.get_feature_names()

In [10]:
print(train_data_features.shape)

(1400, 12183)


## Predicting subreddit using Random Forests + Another Classifier

Train-Test Split is a very important part of predicting unseen data. Train-test split allows us to divide the sample data into two categories: train and test. The data in the train category is the data that our model is fed to make its predictions. The data in the test category is the data our model is exposed to and is expected to make predictions from using the data the model was fed in the train data. In our case, we will be making predictions of what posts belong in the sub-reddit nba. 

In [11]:
## YOUR CODE HERE

x = train_data_features
y = data["Category"]
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(train_data_features, y, test_size=.3, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

First, we will use the logistic regression model to predict the categories of posts.

In [12]:
logistic = linear_model.LogisticRegression()
logistic.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Accuracy Score


Accuracy = (True Positive + True Negative)/(Total Number of Values)

How accurate is the logistic regression model? 

In [13]:
## YOUR CODE HERE
predicted = logistic.predict(x_train)
train_score = logistic.score(x_train, y_train)
score = logistic.score(x_test, y_test)
print(score)

0.9619047619047619


While the logistic regression model holds value in correctly predicting which category the reddit post belongs in, it is still important to test other statistical models. One such model would be the Random Forest Classification Model. The Random Forest Classification Model is an ensemble model that creates multiple trees using different combinations of features of the reddit posts such as the title or the reddit text content and at the end chooses the best performing tree. We want to compare the accuracy score of a random forest classification model with the logistic regression model. 

In [40]:
## YOUR CODE HERE
param_grid = { 
    'n_estimators': [2, 7],
    'max_features': ['auto', 'sqrt', 'log2']
}
gs = GridSearchCV(RandomForestClassifier(), param_grid = param_grid)
gs.fit(x_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9285714285714286
{'max_features': 'auto', 'n_estimators': 7}


Looking at the accuracy scores of the logistic regression model and the random forest classifier, we can clearly see that the logistic regression model is better able to correctly predict the categories of the posts. In this case, the logistic regression model is better able to predict the posts that belong in the nba subreddit.

Looking at two statistical models is great but we should try another model. This model is called a pipeline. Pipelining allows you to incorporate multiple models to help the model accurately predict the categories of reddit posts. In this case, I used a combination of selectkbest and randomforestclassifier to predict categories of posts. 

In [41]:
## YOUR CODE HERE
select = sklearn.feature_selection.SelectKBest(k=100)
clf = sklearn.ensemble.RandomForestClassifier()
steps = [('feature_selection', select),
        ('random_forest', clf)]
parameters = {"feature_selection__k":[100, 200], 
              "random_forest__n_estimators":[50, 100, 200],
              "random_forest__min_samples_split":[2, 3, 4, 5, 10]}
pipeline = sklearn.pipeline.Pipeline(steps)


pipeline.fit(x_train, y_train)

y_prediction = pipeline.predict(x_test)
### test your predictions using sklearn.classification_report()
report = sklearn.metrics.classification_report(y_test, y_prediction)
### and print the report
print(report)

             precision    recall  f1-score   support

          0       0.94      0.97      0.95       107
          1       0.97      0.93      0.95       103

avg / total       0.95      0.95      0.95       210



/anaconda2/envs/dsi/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    3     5     9 ... 12179 12181 12182] are constant.
  UserWarning)
/anaconda2/envs/dsi/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/anaconda2/envs/dsi/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Surprisingly, logistic regression continues to have the highest accuracy though the pipeline's accuracy of 95% is very strong as well. However, let's try two more models. I will choose the GradientBoostingClassifer and ExtraTreesClassifier. 

Gradient Boosting Classifier boosts the performance of a simple base-learner by iteratively shifting the focus towards problematic observations that are difficult to predict. That is, gradient boosting classifier identifies difficult observations by large residuals calculated from previous cycles. 

Extra Trees Classifier creates many trees of various segments of the data, and uses averaging to reduce error rate. 

In [42]:
## YOUR CODE HERE
param_grid = { 
    'n_estimators': [2, 7],
    'max_features': ['auto', 'sqrt', 'log2']
}
gs = GridSearchCV(GradientBoostingClassifier(), param_grid = param_grid)
gs.fit(x_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9204081632653062
{'max_features': 'auto', 'n_estimators': 7}


In [43]:
## YOUR CODE HERE
param_grid = { 
    'n_estimators': [2,7],
    'max_features': ['auto', 'sqrt', 'log2']
}
gs = GridSearchCV(ExtraTreesClassifier(), param_grid = param_grid)
gs.fit(x_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9173469387755102
{'max_features': 'sqrt', 'n_estimators': 7}


# Executive Summary
---
Put your executive summary in a Markdown cell below.

Make three paragraphs long. 

Reddit receives so many posts of a daily basis, and it is so crucial that Reddit successfully identifies the correct category of each post. Posts put in the correct category increases user engagement as it allows more users to share their knowledge with like-minded folks. While having volunteers or employees can also successfully categorize posts in their respective categories, it is a very difficult task to keep up with the large number of posts Reddit receives on a daily basis and also difficult to do so quickly. Luckily for you, my work will correctly put posts in the respective categories and ensure that users are always up to date on the latest information about the subreddit of their choice. 

In this data analysis report, I showcased how to extract NBA reddit posts and successfully identify future potential NBA reddit posts. I also gathered data from other subreddits to ensure that my predictions can differentiate basketball posts from other categories. As proof of how strong my predictions are, the data shows that my predictions can correctly identify 90+% of reddit posts with nba content as part of the nba subreddit. With such success, my work can take on other reddit posts and correctly identify those posts in their respective categories and in doing so, my work can save Reddit much time and money. 